# Exercícios banco

Segue abaixo uma descrição das coleções a serem utilizadas nas tarefas seguintes:

Coleções presentes no banco de `analytics`:

* `establishments`: Coleção que armazena dos documentos dos estabelecimentos;
* `page`: Coleção que armazena os documentos relacionados a _page_ do estabelecimento;
* `messages`: Coleção que armazena as mensagens personalizadas do bot;
* `intentblocks`: Coleção que armazena as intenções do bot bloqueadas de serem respondidas por cada estabelecimento;
* `orders`: Coleção que armazena objetos de pedidos dos clientes dos estabelecimentos;
* `bills`: Coleção que armazena objetos referentes as faturas dos estabelecimentos no sistema;
* `establishmentunits`: Coleção que armazena 
* `clientnps`: Coleção que armazena as respostas da pesquisa de NPS realizadas com clientes dos estabelecimentos
* `clientcoupons`: Coleção que armazena cupons retirados pelos clientes dos estabelecimentos
* `clientgifts`: Coleção que armazena os presentes ganhos pelos clientes no plano de fidelidade
* `modules`: Coleção que armazena os módulos do sistema
* `pagemodules`: Coleção que armazena os módulos que os estabelecimentos têm, e se eles estão ou não ativos assim como a data de contração.
* `item_prices`: Coleção que armazena os preços dos itens em cada dia da semana dos itens dos estabelecimentos
* `clients`: Coleção que armazena as informações dos clientes

Coleções presentes no banco `messages` (banco utilizado também para métricas:

**Banco principal**: `messages-bot-production`

* `messages`: Coleção que armazena as mensagens recebidas;


**Importante**: 
* Todas as operações que pedirem informações de um estabelecimento específico, deveremos utilizar a seguinte referência `page` para o objetivo: `ObjectId("5a0df4fc1e8bf00dcccccf7c")`
* Todos os resultados obtidos devem ser obtidos **somente utilizando consultas ao banco**
* Caso seja necessário, o repositório que guarda as *models* de cada uma das coleções está no seguinte link: https://github.com/githubanotaai/models
  

## Resgaste todos os estabelecimentos ativos da plataforma 

Estabelecimentos ativos são aqueles que possuem valor `True` para a propriedade `sign.active`

## Resgate todos os estabelecimentos que entraram na plataforma antes de agosto de 2021

Podemos considerar a data de criação do objeto (somente neste exemplo) para induzir a data de entrada (`createdAt`)

## Resgate quantos clientes tem o estabelecimento especificado

## Resgate quantos clientes realizaram pedido no estabelecimento especificado


## Resgate quais são as intenções bloqueadas do estabelecimento especificado

## Resgate quantos estabelecimentos possuem a intenção `observation` bloqueada independente do canal

## Resgate todos os estabelecimentos que tiveram valor de fatura superior a 200 reais no último mês

## Resgate a quantidade de pedidos relativa a cada possível `check` _(status do pedido)_ do estabelecimento selecionado

* `check` é o campo que corresponde ao status do pedido;

Por exemplo, para cada valor de `check` possível, deveremos retornar a quantidade de pedidos dentro de cada um dos possíveis conjuntos.

## Resgate a quantidade de mensagens existentes no banco que contém cada uma das palavras `pedido` e cardápio

O objetivo é saber quantas mensagens tiveram a palavra pedido e quantas tiveram a palavra cardápio

## Resgate **somente** as informações de cidade e estado dos estabelecimentos ativos

Essas informações estão dentro de `establishmentunits`, dentro da propriedade `address`.


## Resgate a quantidade de mensagens recebidas para cada intenção no desde o começo do dia de ontem

Observação: se a intenção foi igual a mensagem, a mesma deve ser desconsiderada

## Obtenha uma lista dos clientes de acordo com cada nota dada para o NPS abaixo de 7 desde o dia 1 de dezembro de 2021

## Resgate o total de cada uma notas NPS obtidas para cada mês desde agosto de 2021

## Retorne as quantidades relativas a cada um dos métodos de pagamento utilizados pelos estabelecimentos

Quantos trabalham com cada tipo de método de pagamento?

Realize uma contagem dos tipos de cartão de crédito diferentes para sabermos qual é o mais utilizado

Observação: 

MASTERCARD, mastercard, MASTER CARD e master card deverão ser contabilizados como um só tipo


## Retorne os 10 estabelecimentos em que os clientes mais retiraram cupons

## Retorne os 10 estabelecimentos em que os clientes mais retiraram prêmios do programa de fidelidade

Observação: Explore os documentos da coleção para descobrir quando um prêmio já foi retirado

## Retorne a quantidade de prêmios ainda a serem retirados por clientes de cada um dos estabelecimentos **com plano de fidelidade ativo**

## Retorne todos os clientes que faltam apenas um ponto para que eles consigam ganhar o prêmio do plano fidelidade

Só deveremos considerar aqueles que possuem o plano de fidelidade ativo

## Retorne o cliente que mais realizou ações durante um atendimento no último dia

## Retorne os 10 estabelecimentos que possuem mais itens **disponíveis** no cardápio

Quando um item não está disponível, a propriedade `deleted` dele está **setada** como `True`

## Quais são os 5 estabelecimentos com maior quantidade de preços promocionais disponíveis?

Preços promocionais são aqueles que contém a propriedade `is_promotion` igual a `True`, dentro do preço incluso no objeto dos preços por dia da semana (`week`), na coleção `item_prices`

## Qual a quantidade em meses que cada um dos estabelecimentos ativos estão na plataforma?

Para este exemplo (atenção, somente para este, no cenário real isso "não poderia" ser feito dessa forma), considere a data de entrada como a data de criação do documento de cada um dos estabelecimentos `createdAt`.

## Quais é a frase mais comum de cada uma das intenções do bot?

Observação: se a intenção foi igual a mensagem, esta mensagem deve ser desconsiderada
Observação: considerar apenas mensagens enviadas no dia de ontem

## Quantos estabelecimentos ativos temos em cada uma das regiões do país?